<a href="https://colab.research.google.com/github/Manuelamejiah/Business-intelligence-/blob/main/Reto_2_%C3%81rboles_de_decisi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Árbol de decisión**

In [ ]:
import pandas as pd
import numpy as np

#librerias arbol
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix

#conectar al drive -> vamos a tener los archivos disponibles
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. **Se carga la base de datos**

In [ ]:
nxl='/content/drive/MyDrive/ANALÍTICA DE NEGOCIOS /2. CollegeAdmisions_Int_M.xlsx' #NXL SIGNIFICA NAME EXCEL
XDB=pd.read_excel(nxl, sheet_name=0)
XDB.head(100)
total_datos = XDB.size
print(f"El número total de datos en el dataframe es: {total_datos}")

El número total de datos en el dataframe es: 208068


2. **Eliminamos datos fantantes y selecionamos variables a utilizar en el análisis**

In [ ]:
XDB = XDB.dropna()
print('Número total de datos después de limpieza:', len(XDB))

Número total de datos después de limpieza: 1528


In [ ]:
XDB["Gender"] = XDB["Gender"].map({"F": 0, "M": 1})
XDB["Admitted"] = XDB["Admitted"].map({"No": 0, "Yes": 1})

# Seleccionar las variables
XD = XDB[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
YD = XDB[['Admitted']]

A continuación, se hará una breve descripción de las variables seleccionadas para el análisis.
Es importante tener en cuenta que la base de datos cuenta con 1528 datos después de una limpieza y que los estudiantes serán ddescritos en función de 7 variables

* Edu_Parent1:Indica el nivel educativo alcanzado por el padre o tutor 1
* Edu_Parent1:Indica el nivel educativo alcanzado por el padre o tutor 2
* Gender: Indica el género del estudiante. En este caso, para femenino se identificará como 0 y masculino como 1
* White: Indica si el estudiante se identifica como blanco. 0 será 'NO' y 1 será 'SI'
* Asian: Indica si el estudiante se identifica como Asiático. 0 será 'NO' y 1 será 'SI'
* HSPGA:  Promedio de calificaciones del estudiante en la escuela secundaria.
* SAT/ACT: Puntaje que obtuvo el estudiante en los exámenes estandarizados SAT O ACT.

Finalmente tendremos la variable de salida,
* Admitted: determina si el estudiante fue admitido o no en la universidad


3. **Implementación del modelo**

In [ ]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4)
mar.fit(XD,YD)

#se procede con la gráfica
from pydotplus import graph_from_dot_data #me crea la gráfica
from sklearn.tree import export_graphviz #me exporta a png, jpg, tiff

ve= ['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']
dot_data=export_graphviz(mar,feature_names=ve)
graph_from_dot_data(dot_data).write_png('2. ArbolDeDecisión_M.png')


True

In [ ]:
num_nodos_puros = sum(mar.tree_.impurity == 0)
print(f"El número de nodos puros en el árbol es {num_nodos_puros}  (Gini = 0,0):")

El número de nodos puros en el árbol es 7  (Gini = 0,0):


In [ ]:
# prompt: count the amount of admitted students in the column "admitted"

# Assuming YD is a pandas Series or DataFrame column
num_admitidos = YD['Admitted'].sum()  # Sum the values in the 'Admitted' column
print(f"El número de estudiantes admitidos es {num_admitidos}")


El número de estudiantes admitidos es 1303


4. **Se procede con el pronóstico de la admisión para los estudiantes en la   base de datos**

In [ ]:
XDB0=pd.read_excel(nxl, sheet_name=0)
XDB0.head(100)
#vamos a clasificar los que están en la hoja 1 con los de pronóstico de la hoja 2
#Se crea la bd a pronosticar
XDB0["Gender"] = XDB0["Gender"].map({"F": 0, "M": 1})
XD0=XDB0[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
YD0=XDB0[['Admitted']]

filas = XDB0[(XDB0['Admitted'] == 'No') | (XDB0['Admitted'].isna())].index
XD0=np.array(XD0)
XD0=XD0[filas]

#le calculamos la admisión a los estudiantes
y_pred=mar.predict(XD0)
print(y_pred)

#se crea el informe en excel
df=pd.DataFrame(np.column_stack((filas, XD0,y_pred)))
#filas donde estan los estudiantes que no fueron admitidos, XD0 los datos de esas personas, y_pred la decisión
df.columns=['Solicitante', 'Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT','Admitted']
df.to_excel("Admisión_universidad.xlsx")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


[1 1 1 ... 0 1 0]


4. **Evaluar el comportamiento del modelo**

In [ ]:
ydp=mar.predict(XD) #pronóstico sobre la bd original
cm=confusion_matrix(YD,ydp)
print("La matriz de confusión es:\n",cm)
VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]


#Métricas de desempeño
EX=(VP+VN)/(VP+VN+FP+FN) #Exactitud
print('La exactitud del modelo es:\n',EX)

TE=(FN+FP)/(VP+VN+FP+FN) #Tasa de error:
print('La tasa de error del modelo es:\n',TE)

SE=VP/(VP+FN) #Sensibilidad
print('La sensibilidad del modelo es:\n',SE)

SP=VN/(VN+FP) #Especificidad
print('La especificidad del modelo es:\n',SP)

PR=VP/(VP+FP) #Precisión
print('La precisión del modelo es:\n',PR)

PRNeng=VN/(VN+FN) #Precisión Negativa
print('La precisión negativa del modelo es:\n',PRNeng)

La matriz de confusión es:
 [[  20  205]
 [   2 1301]]
La exactitud del modelo es:
 0.8645287958115183
La tasa de error del modelo es:
 0.1354712041884817
La sensibilidad del modelo es:
 0.9984650805832693
La especificidad del modelo es:
 0.08888888888888889
La precisión del modelo es:
 0.8638778220451527
La precisión negativa del modelo es:
 0.9090909090909091


# **Análisis de resultados y del modelo**

La matriz de confusión muestra una predominancia en los valores verdaderos positivos (1301), seguido de los valores falsos positivos (205), frente a un total de 1528 datos después de eliminar los valores atípicos. Se destaca que el modelo identificó correctamente 1301 estudiantes admitidos y 20 no admitidos, y se tuvo un total de 1303 admitidos y 225 no admitidos.

Inicialmente, el modelo cuenta con una exactitud del 86% aproximadamente, que nos indica el porcentaje de estudiantes que fueron o no admitidos que el modelo predijo correctamente. Con respecto a esto, vemos el modelo idetificó erroneamete solo el 13,4% de los estudiantes.

Por otro lado, con la medida de sensibilidad vemos que el modelo identificará correctamente a los estudiantes realmente admitidos en un 99,8%. Pero, al momento de identificar a aquellos que no fueron admitidos, los identificará correctamente solo en un 8%.

Esto nos dice que el modelo tiene un un buen comportamiento solo identificando los estudiantes admitidos, más no aquellos que no fueron admitidos, y esto podría representar un problema si se necesita una alta precisión identificando estudiantes no admitidos.
